In [1]:
# Thank you to https://stmorse.github.io/journal/spotify-api.html for getting started
import numpy as np
import pandas as pd # csv and dataframes
import requests # Spotify API
import config_secrets # Retrieve my secrets from this file

# Secrets from developer.spotify.com account
# You would have to change this
CLIENT_ID = config_secrets.CLIENT_ID
CLIENT_SECRET = config_secrets.CLIENT_SECRET

# How to get a token
AUTH_URL = 'https://accounts.spotify.com/api/token'

# Where to send spotify GET requests
BASE_URL = 'https://api.spotify.com/v1/'

In [2]:
# POST to get our access token
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET
})

auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']

# Include this in every GET request to make sure we have access
headers = {'Authorization': f"Bearer {access_token}"}

In [3]:
### CHANGE THIS ###############
playlist_name = "Taylor Swift"
playlist_id = "4eHM58IENfJBe3XL9OIvjV"
###############################

next_request = f"{BASE_URL}playlists/{playlist_id}/tracks"
chunks = []
while next_request != None:
    r_playlist = requests.get(next_request, headers=headers)
    chunks.append(pd.json_normalize(r_playlist.json()['items']))
    # This will contain a valid request if there are more tracks
    # or None if we have all the tracks in the playlist
    next_request = r_playlist.json()['next']

df_playlist = pd.concat(chunks)
df_playlist.info()

<class 'pandas.core.frame.DataFrame'>
Index: 254 entries, 0 to 53
Data columns (total 40 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   added_at                            254 non-null    object
 1   is_local                            254 non-null    bool  
 2   primary_color                       0 non-null      object
 3   added_by.external_urls.spotify      254 non-null    object
 4   added_by.href                       254 non-null    object
 5   added_by.id                         254 non-null    object
 6   added_by.type                       254 non-null    object
 7   added_by.uri                        254 non-null    object
 8   track.album.album_type              254 non-null    object
 9   track.album.artists                 254 non-null    object
 10  track.album.available_markets       254 non-null    object
 11  track.album.external_urls.spotify   254 non-null    object
 12  

In [4]:
# Get audio features described at the link below (up to 100 songs at a time)
# https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features
#r_features = requests.get(BASE_URL + 'audio-features?ids=' + ','.join(df_playlist['track.id']), headers=headers)
#df_features = pd.json_normalize(r_features.json()['audio_features'])
#df_features.info()

track_groups = np.array_split(df_playlist['track.id'], len(df_playlist) // 100 + 1)
chunks = []
for group in track_groups:
    r_features = requests.get(BASE_URL + 'audio-features?ids=' + ','.join(group), headers=headers)
    chunks.append(pd.json_normalize(r_features.json()['audio_features']))

df_features = pd.concat(chunks)
df_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 254 entries, 0 to 83
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      254 non-null    float64
 1   energy            254 non-null    float64
 2   key               254 non-null    int64  
 3   loudness          254 non-null    float64
 4   mode              254 non-null    int64  
 5   speechiness       254 non-null    float64
 6   acousticness      254 non-null    float64
 7   instrumentalness  254 non-null    float64
 8   liveness          254 non-null    float64
 9   valence           254 non-null    float64
 10  tempo             254 non-null    float64
 11  type              254 non-null    object 
 12  id                254 non-null    object 
 13  uri               254 non-null    object 
 14  track_href        254 non-null    object 
 15  analysis_url      254 non-null    object 
 16  duration_ms       254 non-null    int64  
 17  tim

In [5]:
# Join the original table with the data we got from the Spotify API
data = df_playlist.join(df_features.set_index('id'), on='track.id', how='inner')
data.reset_index(drop=True)
data.info() # Look at those columns

<class 'pandas.core.frame.DataFrame'>
Index: 254 entries, 0 to 53
Data columns (total 57 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   added_at                            254 non-null    object 
 1   is_local                            254 non-null    bool   
 2   primary_color                       0 non-null      object 
 3   added_by.external_urls.spotify      254 non-null    object 
 4   added_by.href                       254 non-null    object 
 5   added_by.id                         254 non-null    object 
 6   added_by.type                       254 non-null    object 
 7   added_by.uri                        254 non-null    object 
 8   track.album.album_type              254 non-null    object 
 9   track.album.artists                 254 non-null    object 
 10  track.album.available_markets       254 non-null    object 
 11  track.album.external_urls.spotify   254 non-null   

In [6]:
data.head() # Look at the first few rows of data

,added_at,is_local,primary_color,added_by.external_urls.spotify,added_by.href,added_by.id,added_by.type,added_by.uri,track.album.album_type,track.album.artists,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,2020-08-17T17:52:38Z,False,None,https://open.spotify.com/user/katiekaminski1115,https://api.spotify.com/v1/users/katiekaminski...,katiekaminski1115,user,spotify:user:katiekaminski1115,album,[{'external_urls': {'spotify': 'https://open.s...,...,0.0,0.1210,0.425,76.009,audio_features,spotify:track:0Om9WAB5RS09L80DyOfTNa,https://api.spotify.com/v1/tracks/0Om9WAB5RS09...,https://api.spotify.com/v1/audio-analysis/0Om9...,232107,4
1,2020-08-17T17:52:38Z,False,None,https://open.spotify.com/user/katiekaminski1115,https://api.spotify.com/v1/users/katiekaminski...,katiekaminski1115,user,spotify:user:katiekaminski1115,album,[{'external_urls': {'spotify': 'https://open.s...,...,0.0,0.0962,0.821,105.586,audio_features,spotify:track:32mVHdy0bi1XKgr0ajsBlG,https://api.spotify.com/v1/tracks/32mVHdy0bi1X...,https://api.spotify.com/v1/audio-analysis/32mV...,173067,4
2,2020-08-17T17:52:38Z,False,None,https://open.spotify.com/user/katiekaminski1115,https://api.spotify.com/v1/users/katiekaminski...,katiekaminski1115,user,spotify:user:katiekaminski1115,album,[{'external_urls': {'spotify': 'https://open.s...,...,0.0,0.1190,0.289,99.953,audio_features,spotify:track:7zMcNqs55Mxer82bvZFkpg,https://api.spotify.com/v1/tracks/7zMcNqs55Mxe...,https://api.spotify.com/v1/audio-analysis/7zMc...,203040,4
3,2020-08-17T17:52:38Z,False,None,https://open.spotify.com/user/katiekaminski1115,https://api.spotify.com/v1/users/katiekaminski...,katiekaminski1115,user,spotify:user:katiekaminski1115,album,[{'external_urls': {'spotify': 'https://open.s...,...,0.0,0.3200,0.428,115.028,audio_features,spotify:track:73OX8GdpOeGzKC6OvGSbsv,https://api.spotify.com/v1/tracks/73OX8GdpOeGz...,https://api.spotify.com/v1/audio-analysis/73OX...,199200,4
4,2020-08-17T17:52:38Z,False,None,https://open.spotify.com/user/katiekaminski1115,https://api.spotify.com/v1/users/katiekaminski...,katiekaminski1115,user,spotify:user:katiekaminski1115,album,[{'external_urls': {'spotify': 'https://open.s...,...,0.0,0.1230,0.261,175.558,audio_features,spotify:track:7an1exwMnfYRcdVQm0yDev,https://api.spotify.com/v1/tracks/7an1exwMnfYR...,https://api.spotify.com/v1/audio-analysis/7an1...,239013,4


In [9]:
data.iloc[210]

added_at                                                           2022-10-21T17:25:36Z
is_local                                                                          False
primary_color                                                                      None
added_by.external_urls.spotify          https://open.spotify.com/user/katiekaminski1115
added_by.href                         https://api.spotify.com/v1/users/katiekaminski...
added_by.id                                                           katiekaminski1115
added_by.type                                                                      user
added_by.uri                                             spotify:user:katiekaminski1115
track.album.album_type                                                            album
track.album.artists                   [{'external_urls': {'spotify': 'https://open.s...
track.album.available_markets         [AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...
track.album.external_urls.spotif

In [10]:
data.to_csv(f"data/{playlist_name}.csv",index=False)